In [1]:
import  pandas as pd
import numpy as np
import re
import datetime
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


In [2]:
df = pd.read_csv("data.csv",index_col=0)

In [3]:
df.head()


,ALICE,BTC,DYDX,ADA,BNB,QTUM,SUSHI,SHIB,ALGO,CHR,...,LTC,MATIC,AAVE,DOT,LINK,NEAR,SOL,AUDIO,RAY,FTM
2021-10-10 16:35:00,NaN,55229.54,NaN,NaN,NaN,NaN,NaN,0.000026,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,153.56,NaN,NaN,NaN
2021-10-10 16:36:00,12.50,55222.47,24.677,2.233,417.4,13.563,10.766,0.000026,1.8263,NaN,...,179.3,1.312,300.1,35.51,26.61,7.651,153.55,2.047,9.962,2.1750
2021-10-10 16:37:00,12.48,55220.76,24.658,2.232,417.5,13.558,10.770,0.000026,1.8244,NaN,...,179.2,1.311,299.9,35.51,26.60,7.653,153.49,2.045,9.959,2.1711
2021-10-10 16:38:00,12.51,55281.82,24.698,2.233,417.7,13.567,10.780,0.000026,1.8264,NaN,...,179.4,1.312,300.0,35.56,26.64,7.637,153.74,2.046,9.975,2.1789
2021-10-10 16:39:00,NaN,55281.74,24.800,2.232,417.7,13.566,10.780,0.000026,1.8273,0.3317,...,179.2,1.313,300.0,35.55,26.66,7.644,153.51,2.047,NaN,2.1768


In [4]:


def cut_data(df, n, quantity=0):
    try:
        start = re.sub(":\d\d:", ":00:", str(df.index[0]))
        bins = pd.date_range(start, df.index[-1], freq=f'{n}Min')
        dti = pd.DatetimeIndex([bins[-1] + datetime.timedelta(minutes=n)])
        bins = bins.append(dti)
        cats = pd.cut(pd.to_datetime(df.index), bins=bins, labels=bins[:-1], right=False)
        df["cats"] = cats
        df1 = df.groupby(["cats"]).apply(lambda i: i.iloc[-1] if len(i) > 0 else None)
        df1.fillna(method='bfill', inplace=True)
        df1.index = cats.categories
        df1.drop('cats', axis=1, inplace=True)

        return df1 if quantity == 0 else df1.tail(quantity)
    except Exception as e:
        df.to_csv("fuck.csv")
        print(df.head(5))
        print(df.tail(5))
        print(f"cut data {n}", start, df.index[-1])
        print(repr(e))


eps = np.round(np.linspace(1,5,21),1)
def reset(data,n, slid, crypto, offset=True):
    if offset:
        df = cut_data(data.tail(n * 200), n)
    else:
        df = cut_data(data, n)
    item = df.tail(slid)
    item.fillna(method="ffill", inplace=True)
    item.fillna(method="bfill", inplace=True)
    scale = StandardScaler()
    t = scale.fit_transform(item)
    # print(t.shape,len(data.columns),data.columns)
    tdata = pd.DataFrame(data=t, columns=df.columns)
    labs = {}
    for ep in eps:
        model = DBSCAN(eps=ep, min_samples=5).fit(tdata.transpose())
        if set(model.labels_).__len__() >1:
            labels = pd.Series(model.labels_[model.labels_!=-1])
            lab = labels.value_counts()
            for c in lab.index:
                labs[c] = (tdata.transpose()[[True if x == c else False for x in model.labels_]]).transpose()
            
            break
    gp,gdata = sorted(labs.items(),key=lambda x:x[-1].shape[1])[0]
    gdata.index = item.index

    print(f"find density group at {ep} ,group {gdata.shape}")
    return gdata

# def loop_reset():
    

In [19]:
gdata = reset(df,15,20,"ETH",False)

find density group at 1.0 ,group (20, 25)


d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [20]:
gdata

,ALICE,BTC,DYDX,ADA,BNB,QTUM,SUSHI,SHIB,ALGO,CHR,...,EOS,BCH,LTC,MATIC,AAVE,LINK,NEAR,SOL,RAY,FTM
2021-11-16 14:30:00,0.361873,0.311666,0.371468,0.315624,0.182971,0.435463,0.569611,0.391827,0.391287,0.553986,...,0.722355,0.563868,0.478428,0.573038,0.657350,0.170679,0.599785,0.338344,0.708265,0.410870
2021-11-16 14:45:00,0.661354,0.712714,0.653161,0.583722,0.514758,0.735580,0.569611,0.570870,0.678157,0.942302,...,0.722355,0.717720,0.810369,0.685399,0.958969,0.801450,0.779324,0.556734,0.827336,0.579055
2021-11-16 15:00:00,0.661354,0.660104,0.646290,0.608094,0.465966,0.806196,0.557431,0.612187,0.630011,0.844316,...,0.610650,0.717720,0.680479,0.685399,0.958969,0.393304,0.691076,0.425057,0.650783,0.638854
2021-11-16 15:15:00,0.636397,0.605786,0.609647,0.486232,0.514758,0.673791,0.622390,0.529552,0.567822,0.927786,...,0.573416,0.763875,0.723775,0.713489,0.941226,0.560273,0.611957,0.505348,0.650783,0.667508
2021-11-16 15:30:00,0.461700,0.234763,0.426432,0.437487,0.280555,0.550214,0.748248,0.460690,0.529707,0.514065,...,0.433785,0.594638,0.680479,0.516858,0.550897,0.356200,0.615000,0.357613,0.556348,0.615183
2021-11-16 15:45:00,0.561527,0.666191,0.664612,0.583722,0.505000,0.832676,0.748248,1.032248,0.656090,0.800766,...,0.657194,0.717720,0.795936,0.713489,0.817031,0.745794,0.767152,0.614543,0.556348,0.650066
2021-11-16 16:00:00,0.536570,0.622107,0.602777,0.754329,0.700169,0.626714,0.679229,0.798116,0.732322,0.913269,...,0.657194,0.671564,0.723775,0.601128,0.817031,0.690137,0.721506,0.370460,0.761642,0.581546
2021-11-16 16:15:00,0.486657,0.552358,0.481397,0.705584,0.641618,0.450175,0.533071,0.722367,0.531713,0.408821,...,0.536181,0.633101,0.608318,0.825849,0.621866,0.467512,0.602828,0.277323,0.761642,0.484373
2021-11-16 16:30:00,0.836051,0.980138,0.898211,0.973682,1.305193,0.782657,0.805088,0.805002,0.924906,0.797137,...,0.852676,0.863879,0.868098,0.825849,0.355732,0.912762,0.748894,0.855414,0.761642,0.838184
2021-11-16 16:45:00,0.910921,1.030220,0.998979,1.022427,1.393019,1.006273,0.849747,0.887637,0.934936,0.695522,...,0.964381,0.825416,0.882530,0.825849,0.355732,1.098283,0.767152,1.186210,0.761642,0.888017


In [12]:
df.shape


(52087, 29)

In [29]:
def loop_main_stream(df):
    result = []
    for item in df.rolling(10):
        item.fillna(method="ffill", inplace=True)
        item.fillna(method="bfill", inplace=True)
        scale = StandardScaler()
        t = scale.fit_transform(item)
        # print(t.shape,len(data.columns),data.columns)
        tdata = pd.DataFrame(data=t, columns=df.columns)
        labs = {}
        for ep in eps:
            model = DBSCAN(eps=ep, min_samples=5).fit(tdata.transpose())
            if set(model.labels_).__len__() >1:
                labels = pd.Series(model.labels_[model.labels_!=-1])
                lab = labels.value_counts()
                for c in lab.index:
                    labs[c] = (tdata.transpose()[[True if x == c else False for x in model.labels_]]).transpose()
#                 print(f"find density group at {ep} ,group {gdata.shape}")
                break
#         print(labs)
        if labs:
            gp,gdata = sorted(labs.items(),key=lambda x:x[-1].shape[1])[0]
            gdata.index = item.index
            result.append(gdata.shape[1])
        else:
            result.append(0)

        
    return result
        

In [31]:
result = loop_main_stream(df.tail(500))

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

...

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
d:\envs\crypt\lib\site-packages\pandas\core\frame.py:4327: SettingW

[0,
 27,
 28,
 28,
 27,
 26,
 26,
 26,
 25,
 25,
 17,
 14,
 10,
 10,
 7,
 7,
 5,
 8,
 10,
 23,
 25,
 26,
 26,
 25,
 27,
 27,
 27,
 17,
 14,
 13,
 13,
 12,
 8,
 5,
 22,
 26,
 25,
 23,
 21,
 21,
 14,
 16,
 14,
 19,
 20,
 17,
 21,
 25,
 25,
 27,
 28,
 28,
 28,
 28,
 21,
 7,
 6,
 11,
 9,
 14,
 18,
 26,
 27,
 22,
 22,
 19,
 14,
 10,
 8,
 5,
 12,
 11,
 10,
 15,
 9,
 13,
 7,
 5,
 7,
 7,
 5,
 13,
 23,
 26,
 27,
 28,
 28,
 28,
 28,
 26,
 21,
 14,
 14,
 13,
 5,
 7,
 13,
 5,
 8,
 10,
 5,
 9,
 5,
 5,
 5,
 7,
 9,
 6,
 7,
 5,
 11,
 15,
 14,
 15,
 18,
 5,
 10,
 9,
 6,
 7,
 6,
 8,
 7,
 9,
 14,
 13,
 8,
 12,
 10,
 15,
 6,
 12,
 5,
 20,
 9,
 11,
 12,
 8,
 10,
 6,
 5,
 6,
 8,
 6,
 10,
 12,
 7,
 6,
 12,
 5,
 5,
 7,
 11,
 10,
 7,
 18,
 25,
 24,
 21,
 15,
 19,
 17,
 15,
 18,
 17,
 18,
 17,
 18,
 20,
 23,
 20,
 13,
 10,
 14,
 16,
 14,
 11,
 5,
 6,
 9,
 10,
 4,
 5,
 5,
 5,
 6,
 5,
 7,
 6,
 10,
 12,
 12,
 6,
 7,
 20,
 5,
 7,
 21,
 23,
 24,
 23,
 20,
 10,
 3,
 13,
 12,
 18,
 20,
 19,
 16,
 16,
 15,
 13,
 19,
 1

In [25]:
td = {1:"aaa",5:"a",2:"ccccc"}
sorted(td.items(),key=lambda x:len(x[-1]))




[(5, 'a'), (1, 'aaa'), (2, 'ccccc')]

In [26]:
df.rolling()

TypeError: rolling() missing 1 required positional argument: 'window'

In [9]:
a = pd.Series(range(9))

In [10]:
a

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
dtype: int64

In [11]:
a.rolling(2).mean()

0    NaN
1    0.5
2    1.5
3    2.5
4    3.5
5    4.5
6    5.5
7    6.5
8    7.5
dtype: float64